In [2]:
# must use python3 to work
import bz2

# read first n lines of zipped file and make new file
# add multithreading
read_file = 'lichess_db_standard_rated_2021-09.pgn.bz2'
write_file = 'lichess_db_standard_rated_2021-09_subset.pgn'

# read zipped file and write subset as unzipped file
with bz2.BZ2File(read_file, "rb") as bzfin:
    with open(write_file, "wb") as fout:
        for i, line in enumerate(bzfin):
            if i % 1000000 == 0: print(i)
            if i == 50000000: break
            fout.write(line)

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000


In [24]:
# count eval games from ~2.5m games
j = 0
with open('lichess_db_standard_rated_2021-09_subset.pgn', 'r') as f:
    for i, line in enumerate(f):
        if '%eval' in line:
            j += 1
            pass
        if i == 50000000:
            break
            
print(j)

# 200,000 of 2.5m games - 1 in 12.5 games has an eval

207045


In [184]:
# record tuple with location of lines for games with evals
import itertools

j = 0
rows_to_read = []
with open('lichess_db_standard_rated_2021-09_subset.pgn', 'r') as f:
    with open('lichess_db_standard_rated_2021-09_eval_lines_subset.pgn', "w") as eval_out:
        for i, line in enumerate(f):
            if '%eval' in line: 
                # create file with only eval lines
                eval_out.write(line)
                if rows_to_read == []:
                    rows_to_read.append((last_game_end+2,j+1))
                else:
                    rows_to_read.append((last_game_end,j))
                j = 0
            if line[0:3] in ('1. ', '1-0', '0-1', '1/2'):
                last_game_end = j
            if i % 1000000 == 0: print(i)
            if i == 50000000: break
            j += 1

# create eval file
with open('lichess_db_standard_rated_2021-09_subset.pgn', 'r') as fin:
    with open('lichess_db_standard_rated_2021-09_eval_subset.pgn', "w") as fout:
        for tupl in rows_to_read:
            for line in itertools.islice(fin, tupl[0], tupl[1]):
                fout.write(line)

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000


In [4]:
# Check if game has eval. If not discard.
# If there is a brilliant move, record moves and evals
# If not, record number of moves

# create an unzipped pgn file that is a subset of the recent file
# grab only records with move evals
import json, os
import chess.pgn
import pandas as pd
import re

file = 'lichess_db_standard_rated_2021-09_eval_subset.pgn'
pgn = open(file)

i = 0
game_d = {}

with open(file, 'r') as f:
    while True:
        # games data
        headers = chess.pgn.read_headers(pgn)
        if headers is None:
            break
        if i % 10000 == 0:
            print(i)
#         if i == 10000:
#             break

        game_type = headers['Event'].split(" ")[1]
        game_id = headers['Site'].replace("https://lichess.org/", "")
        utc_date = headers['UTCDate']
        utc_time = headers['UTCTime']
        game_round = headers['Round']
        white = headers['White']
        black = headers['Black']
        result = headers['Result']
        white_elo = headers['WhiteElo']
        black_elo = headers['BlackElo']
        termination = headers['Termination']

        # add a dictionary entry to the final dictionary
        game_d[i] = {'type': game_type, 
                'id': game_id, 
                'utc_date': utc_date,
                'utc_time': utc_time,
                'round': game_round,
                'white': white,
                'black': black,
                'result': result,
                'white_elo': int(white_elo),
                'black_elo': int(black_elo),
                'termination': termination,
                'temp_id': i}
        i += 1
        
# create the dataframe using 'from_dict'
# important to set the 'orient' parameter to "index" to make the keys as rows
games = pd.DataFrame.from_dict(game_d, "index")

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000


In [176]:
import numpy as np
eval_file = 'lichess_db_standard_rated_2021-09_eval_lines_subset.pgn'
move_d = {}

# moves data
with open(eval_file, 'r') as f:
    for i, line in enumerate(f):
        if i % 1000 == 0:
            print(i)
        if i == 10000:
            break
        # seperate the string by " [0-9]+\. " or " [0-9]+\.\.\. "
        split_line = re.split(" [0-9]+\. | [0-9]+\.\.\. ", line)

        for j, move in enumerate(split_line):
            temp_d = {}
            if j == 0: 
                move_value = move.split()[1]
            else: 
                move_value = move.split()[0]

            # checkmate move has no eval value; in this case set to "-"
            try:
                eval_obj = re.search('\%eval (-?[0-9]+\.[0-9]+|#-?[0-9]+)', move).group(0)
            except AttributeError: 
                if '#' not in move:
                    # remove games with 200+ moves where eval is lost
                    for k in range(j):
                        move_d.popitem()
                    break
                eval_obj = '-'
  
            eval_value = eval_obj.replace('%eval ', "")
            if j % 2 == 0:
                white = 1
            else:
                white = 0

            # add a dictionary entry to moves dictionary
            move_d[f'{i},{j}'] = {'temp_id': i,        #'game_id': game_id,
                                'move_number': j+1,
                                'move': move_value,  
                                'eval': eval_value,
                                'white': white
                                }

# moves = pd.DataFrame.from_dict(move_d, "index")

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [177]:
moves = pd.DataFrame.from_dict(move_d, "index") 

In [178]:
# use game_id as join key; drop temp_id
# If I want to merge later: pd.merge(moves[moves['last_move_fcm']==1], games[['id', 'result']], how='left', left_on='game_id', right_on='id')
temp = moves.merge(games[['temp_id','id']], on='temp_id')
temp = temp.rename(columns={'id': 'game_id'})
temp = temp.drop('temp_id', 1)
moves = temp
games = games.drop('temp_id', 1)

/var/folders/_1/fhhdt3rx4b547dqcswnn523w0000gn/T/ipykernel_88645/1862863890.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  temp = temp.drop('temp_id', 1)
/var/folders/_1/fhhdt3rx4b547dqcswnn523w0000gn/T/ipykernel_88645/1862863890.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  games = games.drop('temp_id', 1)


In [179]:
# %lprun -f test test()
moves

,move_number,move,eval,white,game_id
0,1,d4,0.0,1,dsPxOAtT
1,2,e6,0.21,0,dsPxOAtT
2,3,Bf4,0.0,1,dsPxOAtT
3,4,d6,0.48,0,dsPxOAtT
4,5,e3,0.47,1,dsPxOAtT
...,...,...,...,...,...
650559,48,dxe5,-7.14,0,QLx9AT2j
650560,49,fxe5,-7.46,1,QLx9AT2j
650561,50,c4,-6.64,0,QLx9AT2j
650562,51,Qxb6,-6.54,1,QLx9AT2j


In [14]:
# add difference between last eval and current eval
# temp = pd.concat([moves, moves[['eval']].shift()], axis=1)
# temp.columns = ['move_number', 'move', 'eval', 'white', 'game_id', 'last_eval']
moves['last_eval'] = moves.sort_values(by=['move_number'], ascending=True).groupby(['game_id'])['eval'].shift(1)
moves['diff'] = moves['eval'] - moves['last_eval']
moves['diff_adj'] = np.where(moves['white'] == 1, moves['diff'], moves['diff'] * -1)
moves[(abs(moves['last_eval'])<10)&(abs(moves['eval'])<10)].sort_values(by=['diff_adj'], ascending=False).head(25)

In [180]:
moves['next_move_eval'] = moves.sort_values(by=['move_number'], ascending=True).groupby(['game_id'])['eval'].shift(-1)

In [181]:
forced_cm_df = moves[(moves['eval']=="#1")&(moves['white']==0)|(moves['eval']=="#-1")&(moves['white']==1)]

In [182]:
forced_cm_df = pd.merge(forced_cm_df, games[['id', 'result', 'black_elo','white_elo']], how='left', left_on='game_id', right_on='id')

In [183]:
# forced mate elo
forced_cm_df['fm_elo'] = np.where(forced_cm_df['eval']=='#1', forced_cm_df['white_elo'], forced_cm_df['black_elo'])

In [193]:
forced_cm_df['fm_elo_floor'] = np.floor(forced_cm_df['fm_elo']/100.0) * 100

In [164]:
# percent of checkmate in 1s that are completed
pct_fcm = forced_cm_df[(forced_cm_df.next_move_eval == '-') | (forced_cm_df.next_move_eval.isna()) &
                       ((forced_cm_df.result == '1-0') & (forced_cm_df.white == 0) | 
                       (forced_cm_df.result == '0-1') & (forced_cm_df.white == 1))]['id'].count() / forced_cm_df['id'].count()
pct_fcm

0.6592902711323764

In [211]:
# subsetting by checkmate in 1 % by elo
fcm_group = pd.DataFrame(forced_cm_df[(forced_cm_df.next_move_eval == '-') | (forced_cm_df.next_move_eval.isna()) & 
             ((forced_cm_df.result == '1-0') & (forced_cm_df.white == 1) | 
             (forced_cm_df.result == '0-1') & (forced_cm_df.white == 0))].groupby(['fm_elo_floor'])['id'].count()) #/ forced_cm_df['id'].count()
fcm_group_total = pd.DataFrame(forced_cm_df.groupby(['fm_elo_floor'])['id'].count())
fcm_group = fcm_group.rename(columns={'id': 'fcm'})
fcm_group_total = fcm_group_total.rename(columns={'id': 'total'})

In [217]:
# percent sucessful forced checkmate by elo floor
pct_fcm_group = fcm_group_total.join(fcm_group, how='outer')
pct_fcm_group['pct'] = pct_fcm_group['fcm'] / pct_fcm_group['total']
pct_fcm_group

,total,fcm,pct
fm_elo_floor,,,
600.0,26,7,0.269231
700.0,48,27,0.562500
800.0,119,60,0.504202
900.0,198,100,0.505051
1000.0,303,171,0.564356
1100.0,376,233,0.619681
1200.0,418,275,0.657895
1300.0,499,308,0.617234
1400.0,495,336,0.678788


In [ ]:
# other ideas include
    # game with the most checkmates in one failed - this will probably be someone promoting a queen out of spite
    # games where a player lost after checkmate in one. highest elo example. 
    # game with player with the highest elo who missed a checkmate in one
    # my rate of checkmates in one accomplished or failed
    # players who with the most games and a perfect checkmate in one pct